# Homework 3 - MapReduce

### DUE: 03/08/2017 before class at 9:30am

In this homework, we are practicing the MapReduce programming paradigm. 

You are required to turn in this notebook as BDM\_HW4.ipynb. You will be asked to complete each task using the accompanied *mapreduce* package (as **mapreduce.py**) using one or more MapReduce "jobs". For each such job (with mr.run()), you are expected to supply a mapper and a reducer as needed. Below are sample usage of the package:

```python
    # Run on input1 using your mapper1 and reducer1 function
    output = list(mr.run(input1, mapper1, reducer1))

    # Run on input2 using only your mapper2, no reduce phase
    output = list(mr.run(input2, mapper2))
    
    # Run on input3 using 2 nested MapReduce jobs
    output = list(mr.run(mr.run(input3, mapper3, reducer3), mapper4, reducer4))
```
    
Please note that the output the mr.run() is always a **generator**. You have to cast it to a list if you'd like to view, index or print it out.

## Task (10 points)

There is only one task in this homework. You are asked to implement the Social Triangle example discussed in class. In particular, given the email dataset, please list all "reciprocal" relationships in the company. Recall that:

If A emails B and B emails A, then A and B is *reciprocal*.

If A emails B but B doesn’t email A, then A and B is *directed*.

**Dataset:** We will use a subset of the open [Enron Email Dataset](https://www.cs.cmu.edu/~./enron/ "Enron Email Dataset"), which contains approximately 10,000 simplified email headers from the Enron Corporation. You can download this dataset from Blackboard as **enron_mails_small.csv**. The file contains 3 columns *Date*, *From*, and *To*. Their description is as follows:

|Column name|Description|
|--|--|
|Date |The date and time of the email, in the format YYYY-MM-DD hh-mm-ss, <br />e.g. "1998-10-30 07:43:00" |
|From |The sender email address, <br />e.g. "mark.taylor@enron.com" |
|To | A list of recipients' email addresses separated by semicolons ';', <br />e.g. "jennifer.fraser@enron.com;jeffrey.hodge@enron.com" |

Note that, we only care about users employed by Enron, or only relationships having email addresses that end with *'@enron.com'*.

The expected output is also provided below. For each reciprocal relationship, please output a tuple consisting of two strings. The first one is always **'reciprocal'**. And the second one is a string showing the name of the two person in the following format: **'Jane Doe : John Doe'**. The names should be presented in the lexical order, i.e. there will not be a 'John Doe : Jane Doe' since 'Jane' is ordered before 'John.

Though the dataset only contains email addresses, not actual names, we're assuming that the email aliases were created based on their name. For example:

|Email Address|Converted Name|
|--|--|
|mark.taylor@enron.com|Mark Taylor|
|alan.aronowitz@enron.com|Alan Aronowitz|
|marc.r.cutler@enron.com|Marc R Cutler|
|hugh@enron.com|Hugh|

Please fill the code block with a series of MapReduce jobs using your own mapper and reducer functions. Be sure to include the naming convention logic into one of your mappers and/or reducers.

In [118]:
import csv
import mapreduce as mr

#<YOUR MAPPERS and REDUCERS>
def mapper1(row):
    for i in row['To'].split(';'):
        if i.split('@')[1] == 'enron.com':
            emailFrom = row['From'].split('.')[0] + " " + row['From'].split('.')[1].split('@')[0]
            emailTo = i.split('.')[0] + " " + i.split('.')[1].split('@')[0]
            yield ((emailFrom, emailTo), 1)

def reducer1(((emailFrom, emailTo), counts)):
#     nameMap = {(emailFrom, emailTo)}
#     return nameMap[(emailFrom, emailTo)]
    return ((emailFrom, emailTo), sum(counts))#,((emailTo, emailFrom), 1 ))

def mapper2(((emailFrom, emailTo), counts)):
    pair = (emailFrom, emailTo)
    oppositePair = (emailTo, emailFrom)
    if pair < oppositePair:
        yield (pair, 1)
    else:
        yield (oppositePair, 1)
    
def reducer2((name, counts)):
    return (name,sum(counts))

    
def mapper3((name, counts)):
    if (counts==2):
        yield ("Reciprocal  -  " + str(name[0]) + ' : ' + str(name[1]))
    
#     fromName = emailFrom.split('@')[0].split('.')
#     if(emailTo.split('@')[1] == 'enron.com'):
#         toName = emailTo.split('@')[0].split('.')        
#         yield ((fromName, toName), 1)
#         yield ((toName, fromName), 1)
        
    
#     if name1 == name2:
#         return(name1, name2)
    #return (((name1, name2), sum(counts)))

# def reducer3(((name1, name2), counts)):
#     if counts>1 and name1!=name2:
#         return (name1, name2)
    
#     newPair = (NameTo, NameFrom)
#     nameMap = { newPair: 0 }
    
    
    #return (((NameFrom, NameTo), counts), ((NameTo, NameFrom), 0))
    
    
    
        
with open('enron_mails_small.csv', 'r') as fi:
    reader = csv.DictReader(fi)
    #output1 = mr.run(reader, mapper1, reducer1) # Get the row's 'From' and 'To' columns
    #output2 = mr.run(output1, mapper2) # Splits output1 into first name and last name for both 'From' and 'To' 
    #output3 = filter(lambda x: x!=None, output2) #Filters the None's out 
    #output4 = list(mr.run(output3, mapper2))
    
    #output = list(mr.run(reader, mapper1, reducer1))
    
    output = list(mr.run(mr.run(mr.run(reader, mapper1, reducer1), mapper2, reducer2), mapper3))
    
    #output =  filter(lambda x: x[1]>1 ,(mr.run((mr.run(reader, mapper1, reducer1)), mapper2, reducer2) ))
    
    
    #output = list(mr.run(mr.run(reader, mapper1, reducer1), ...))

    
print(len(output))
output



35


['Reciprocal  -  brenda whitehead : elizabeth sager',
 'Reciprocal  -  carol clair : debra perlingiere',
 'Reciprocal  -  carol clair : mark taylor',
 'Reciprocal  -  carol clair : richard sanders',
 'Reciprocal  -  carol clair : sara shackleton',
 'Reciprocal  -  carol clair : tana jones',
 'Reciprocal  -  debra perlingiere : kevin ruscitti',
 'Reciprocal  -  drew fossum : susan scott',
 'Reciprocal  -  elizabeth sager : janette elbertson',
 'Reciprocal  -  elizabeth sager : mark haedicke',
 'Reciprocal  -  elizabeth sager : mark taylor',
 'Reciprocal  -  elizabeth sager : richard sanders',
 'Reciprocal  -  eric bass : susan scott',
 'Reciprocal  -  fletcher sturm : greg whalley',
 'Reciprocal  -  fletcher sturm : sally beck',
 'Reciprocal  -  gerald nemec : susan scott',
 'Reciprocal  -  grant masson : vince kaminski',
 'Reciprocal  -  greg whalley : richard sanders',
 'Reciprocal  -  janette elbertson : mark taylor',
 'Reciprocal  -  janette elbertson : richard sanders',
 'Reciproca

In [12]:

# 35
# Out[1]:
# [('recipropcal', 'Brenda Whitehead : Elizabeth Sager'),
#  ('recipropcal', 'Carol Clair : Debra Perlingiere'),
#  ('recipropcal', 'Carol Clair : Mark Taylor'),
#  ('recipropcal', 'Carol Clair : Richard Sanders'),
#  ('recipropcal', 'Carol Clair : Sara Shackleton'),
#  ('recipropcal', 'Carol Clair : Tana Jones'),
#  ('recipropcal', 'Debra Perlingiere : Kevin Ruscitti'),
#  ('recipropcal', 'Drew Fossum : Susan Scott'),
#  ('recipropcal', 'Elizabeth Sager : Janette Elbertson'),
#  ('recipropcal', 'Elizabeth Sager : Mark Haedicke'),
#  ('recipropcal', 'Elizabeth Sager : Mark Taylor'),
#  ('recipropcal', 'Elizabeth Sager : Richard Sanders'),
#  ('recipropcal', 'Eric Bass : Susan Scott'),
#  ('recipropcal', 'Fletcher Sturm : Greg Whalley'),
#  ('recipropcal', 'Fletcher Sturm : Sally Beck'),
#  ('recipropcal', 'Gerald Nemec : Susan Scott'),
#  ('recipropcal', 'Grant Masson : Vince Kaminski'),
#  ('recipropcal', 'Greg Whalley : Richard Sanders'),
#  ('recipropcal', 'Janette Elbertson : Mark Taylor'),
#  ('recipropcal', 'Janette Elbertson : Richard Sanders'),
#  ('recipropcal', 'Liz Taylor : Mark Haedicke'),
#  ('recipropcal', 'Mark Haedicke : Mark Taylor'),
#  ('recipropcal', 'Mark Haedicke : Michelle Cash'),
#  ('recipropcal', 'Mark Haedicke : Richard Sanders'),
#  ('recipropcal', 'Mark Haedicke : Twanda Sweet'),
#  ('recipropcal', 'Mark Taylor : Sara Shackleton'),
#  ('recipropcal', 'Mark Taylor : Tana Jones'),
#  ('recipropcal', 'Michelle Cash : Twanda Sweet'),
#  ('recipropcal', 'Pinnamaneni Krishnarao : Vince Kaminski'),
#  ('recipropcal', 'Richard Sanders : Sara Shackleton'),
#  ('recipropcal', 'Rosalee Fleming : Steven Kean'),
#  ('recipropcal', 'Sara Shackleton : Tana Jones'),
#  ('recipropcal', 'Shirley Crenshaw : Vince Kaminski'),
#  ('recipropcal', 'Stinson Gibner : Vince Kaminski'),
#  ('recipropcal', 'Vasant Shanbhogue : Vince Kaminski')]

In [14]:
string1 = "hello world"
a, b = string1.split(' ')
print a
print b

hello
world
